# Module 2: Initial Price Push (Daily 6 AM Reset)

## Purpose
This module runs once daily at 6 AM Cairo time to:
1. Reset prices for all SKUs based on ABC classification
2. Set initial cart rules based on normal_refill and stddev
3. Apply status-based adjustments (combined_status + yesterday_status)

## Price Setting Logic
- **Zero demand SKUs**: Market minimum price + SKU discount
- **With market data**: A=25th percentile, B=50th, C=75th
- **Without market data**: A=50% margin range, B=75%, C=90%
- **No data SKUs**: Average margin of their category

## Status Adjustment Logic
- Both below On Track: -1 step from target
- Both above On Track: +1 step from target
- Combined lower, Yesterday higher: No action
- Combined higher, Yesterday lower: -1 step from target
- On Track: No action


In [ ]:
# =============================================================================
# IMPORTS AND SETUP
# =============================================================================
import pandas as pd
import numpy as np
from datetime import datetime
import sys
sys.path.append('..')

# Configuration constants
ABC_MARKET_PERCENTILES = {'A': 25, 'B': 50, 'C': 75}
ABC_MARGIN_PERCENTILES = {'A': 50, 'B': 75, 'C': 90}
ABC_CART_STD_MULTIPLIERS = {'A': 1, 'B': 2, 'C': 5}
MIN_CART_RULE = 2
MAX_CART_RULE = 150
MIN_PRICE_REDUCTION_PCT = 0.0025
STATUS_BELOW_ON_TRACK = ['No Data', 'Critical', 'Struggling', 'Underperforming']
STATUS_ABOVE_ON_TRACK = ['Over Achiever', 'Star Performer']
STATUS_ON_TRACK = ['On Track']

# Input/Output files
INPUT_FILE = '../pricing_with_discount.xlsx'
OUTPUT_FILE = f'module_2_output_{datetime.now().strftime("%Y%m%d_%H%M")}.xlsx'
ZERO_DEMAND_SKU_DISCOUNT = 5  # 5% discount

print(f"Module 2: Initial Price Push")
print(f"Run Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Input: {INPUT_FILE}")
print(f"Output: {OUTPUT_FILE}")


In [ ]:
# =============================================================================
# LOAD DATA
# =============================================================================
print("Loading data from Module 1 output...")
df = pd.read_excel(INPUT_FILE)
print(f"Loaded {len(df)} records")
print(f"\nABC Class Distribution:")
print(df['abc_class'].value_counts().to_string() if 'abc_class' in df.columns else "No abc_class column")


In [ ]:
# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def is_below_on_track(status):
    """Check if status is below On Track."""
    return str(status).strip() in STATUS_BELOW_ON_TRACK

def is_above_on_track(status):
    """Check if status is above On Track."""
    return str(status).strip() in STATUS_ABOVE_ON_TRACK

def is_on_track(status):
    """Check if status is On Track."""
    return str(status).strip() in STATUS_ON_TRACK

def calculate_margin(price, wac):
    """Calculate margin from price and WAC."""
    if pd.isna(price) or pd.isna(wac) or price == 0:
        return None
    return (price - wac) / price

def build_price_tiers(row):
    """Build price tiers from market percentiles and margin tiers."""
    tiers = {}
    wac = row.get('wac_p', 0)
    
    # Market-based tiers
    for col, key in [('minimum', 'market_min'), ('percentile_25', 'market_p25'),
                     ('percentile_50', 'market_p50'), ('percentile_75', 'market_p75'),
                     ('maximum', 'market_max')]:
        if pd.notna(row.get(col)):
            tiers[key] = row[col]
    
    # Margin-based tiers
    if wac > 0:
        for i, tier_col in enumerate(['margin_tier_1', 'margin_tier_2', 'margin_tier_3', 
                                       'margin_tier_4', 'margin_tier_5']):
            if pd.notna(row.get(tier_col)):
                margin = row[tier_col]
                if margin < 1:
                    tiers[f'margin_t{i+1}'] = wac / (1 - margin)
    return tiers

print("Helper functions loaded.")


In [ ]:
# =============================================================================
# TARGET PRICE & STATUS ADJUSTMENT FUNCTIONS
# =============================================================================

def get_target_price_for_sku(row):
    """
    Determine target price based on ABC class.
    With market data: A=25th percentile, B=50th, C=75th
    Without market data: A=50% margin range, B=75%, C=90%
    """
    abc_class = str(row.get('abc_class', 'C')).upper()
    wac = row.get('wac_p', 0)
    has_market_data = pd.notna(row.get('minimum')) and pd.notna(row.get('percentile_25'))
    
    if has_market_data:
        percentile_map = {'A': 'percentile_25', 'B': 'percentile_50', 'C': 'percentile_75'}
        target_price = row.get(percentile_map.get(abc_class, 'percentile_50'))
        return target_price, 'market'
    else:
        margin_pct_map = {'A': 0.50, 'B': 0.75, 'C': 0.90}
        margin_pct = margin_pct_map.get(abc_class, 0.75)
        min_margin = row.get('effective_min_margin', row.get('min_boundary', 0.05))
        max_margin = row.get('max_boundary', 0.25)
        
        if pd.notna(min_margin) and pd.notna(max_margin) and wac > 0:
            target_margin = min_margin + (max_margin - min_margin) * margin_pct
            target_price = wac / (1 - target_margin) if target_margin < 1 else wac * 1.1
            return target_price, 'margin'
        return row.get('current_price', wac * 1.15), 'fallback'

def get_status_adjustment(combined_status, yesterday_status):
    """Determine price step adjustment based on status combination."""
    combined_below = is_below_on_track(combined_status)
    combined_above = is_above_on_track(combined_status)
    combined_on = is_on_track(combined_status)
    yesterday_below = is_below_on_track(yesterday_status)
    yesterday_above = is_above_on_track(yesterday_status)
    yesterday_on = is_on_track(yesterday_status)
    
    if combined_on:
        return 0, "On Track - no adjustment"
    if combined_below and (yesterday_below or yesterday_on):
        return -1, f"Both below/on ({combined_status}, {yesterday_status}) - reduce"
    if combined_above and (yesterday_above or yesterday_on):
        return 1, f"Both above/on ({combined_status}, {yesterday_status}) - increase"
    if combined_below and yesterday_above:
        return 0, f"Oscillation prevention - no action"
    if combined_above and yesterday_below:
        return -1, f"Stabilizing - reduce"
    return 0, "Default - no adjustment"

def apply_price_step(base_price, steps, row):
    """Apply price step adjustment."""
    if steps == 0 or pd.isna(base_price):
        return base_price
    wac = row.get('wac_p', 0)
    margin_step = row.get('margin_step', 0.02)
    
    if wac > 0 and pd.notna(margin_step) and margin_step > 0:
        current_margin = (base_price - wac) / base_price if base_price > 0 else 0
        new_margin = max(0.01, min(0.99, current_margin + (steps * margin_step)))
        new_price = wac / (1 - new_margin)
    else:
        new_price = base_price * (1 + steps * 0.02)
    
    if steps < 0:
        new_price = min(new_price, base_price * (1 - MIN_PRICE_REDUCTION_PCT))
    
    commercial_min = row.get('commercial_min_price', row.get('minimum', 0))
    if pd.notna(commercial_min) and commercial_min > 0:
        new_price = max(new_price, commercial_min)
    return round(new_price, 2)

def get_initial_cart_rule(row):
    """Calculate initial cart rule: A=refill+1*std, B=refill+2*std, C=refill+5*std"""
    abc_class = str(row.get('abc_class', 'C')).upper()
    normal_refill = row.get('normal_refill', 5)
    stddev = row.get('refill_stddev', 2)
    current_cart = row.get('cart_rule', normal_refill)
    std_multiplier = ABC_CART_STD_MULTIPLIERS.get(abc_class, 5)
    target_cart = normal_refill + (std_multiplier * stddev)
    new_cart = target_cart if current_cart < target_cart else current_cart
    return max(MIN_CART_RULE, min(MAX_CART_RULE, int(new_cart)))

print("Target price and status functions loaded.")


In [ ]:
# =============================================================================
# MAIN ENGINE: GENERATE INITIAL PRICE PUSH
# =============================================================================

def generate_initial_price_push(row):
    """Generate initial price push action for a single SKU."""
    result = {
        'product_id': row.get('product_id'),
        'warehouse_id': row.get('warehouse_id'),
        'sku': row.get('sku'),
        'brand': row.get('brand'),
        'cat': row.get('cat'),
        'abc_class': row.get('abc_class', 'C'),
        'current_price': row.get('current_price'),
        'wac_p': row.get('wac_p'),
        'stocks': row.get('stocks', 0),
        'combined_status': row.get('combined_status'),
        'yesterday_status': row.get('yesterday_status'),
        'zero_demand': row.get('zero_demand', 0),
        'sensitivity': row.get('sensitivity', row.get('product_sensitivity')),
        'new_price': None,
        'new_cart_rule': None,
        'new_margin': None,
        'current_margin': None,
        'sku_discount': 0,
        'price_source': None,
        'price_reason': None,
    }
    
    wac = row.get('wac_p', 0)
    current_price = row.get('current_price', 0)
    result['current_margin'] = calculate_margin(current_price, wac)
    
    # CASE 1: Zero Demand SKUs - Market minimum + SKU discount
    if row.get('zero_demand', 0) == 1:
        market_min = row.get('minimum')
        if pd.notna(market_min) and market_min > 0:
            result['new_price'] = market_min
            result['price_source'] = 'market_min'
        else:
            result['new_price'] = row.get('margin_tier_1', current_price)
            result['price_source'] = 'margin_min'
        result['sku_discount'] = ZERO_DEMAND_SKU_DISCOUNT
        result['price_reason'] = 'Zero demand - market min + SKU discount'
        result['new_cart_rule'] = get_initial_cart_rule(row)
        result['new_margin'] = calculate_margin(result['new_price'], wac)
        return result
    
    # CASE 2: Get target price based on ABC class
    target_price, price_source = get_target_price_for_sku(row)
    
    # CASE 3: Apply status adjustment
    combined_status = row.get('combined_status', 'No Data')
    yesterday_status = row.get('yesterday_status', 'No Data')
    
    # Handle 'No Data' with stocks as Critical
    if combined_status == 'No Data' and row.get('stocks', 0) > 0:
        combined_status = 'Critical'
    
    steps, reason = get_status_adjustment(combined_status, yesterday_status)
    new_price = apply_price_step(target_price, steps, row)
    
    result['new_price'] = new_price
    result['price_source'] = price_source
    result['price_reason'] = f"ABC={row.get('abc_class')} + {reason}"
    result['new_cart_rule'] = get_initial_cart_rule(row)
    result['new_margin'] = calculate_margin(new_price, wac)
    
    return result

print("Main engine function loaded.")


In [ ]:
# =============================================================================
# EXECUTE MODULE 2
# =============================================================================
print(f"Processing {len(df)} SKUs...")
print("="*60)

results = []
for idx, row in df.iterrows():
    result = generate_initial_price_push(row)
    results.append(result)
    
    if (idx + 1) % 10000 == 0:
        print(f"Processed {idx + 1}/{len(df)} SKUs...")

df_results = pd.DataFrame(results)
print(f"\n✅ Processed {len(df_results)} SKUs")


In [ ]:
# =============================================================================
# SUMMARY
# =============================================================================
print("="*60)
print("MODULE 2 SUMMARY")
print("="*60)

print(f"\nTotal SKUs processed: {len(df_results)}")
print(f"\nBy ABC Class:")
print(df_results['abc_class'].value_counts().to_string())
print(f"\nBy Price Source:")
print(df_results['price_source'].value_counts().to_string())
print(f"\nZero Demand SKUs with discount: {len(df_results[df_results['sku_discount'] > 0])}")

# Price change analysis
df_results['price_change'] = df_results['new_price'] - df_results['current_price']
df_results['price_change_pct'] = (df_results['price_change'] / df_results['current_price'] * 100).round(2)

print(f"\nPrice Change Distribution:")
print(f"  Increases: {len(df_results[df_results['price_change'] > 0])}")
print(f"  Decreases: {len(df_results[df_results['price_change'] < 0])}")
print(f"  No change: {len(df_results[df_results['price_change'] == 0])}")
print(f"\nAvg price change: {df_results['price_change_pct'].mean():.2f}%")


In [ ]:
# =============================================================================
# EXPORT RESULTS
# =============================================================================
output_cols = [
    'product_id', 'warehouse_id', 'sku', 'brand', 'cat', 'abc_class', 'sensitivity',
    'stocks', 'zero_demand', 'combined_status', 'yesterday_status',
    'current_price', 'new_price', 'price_change', 'price_change_pct',
    'wac_p', 'current_margin', 'new_margin',
    'new_cart_rule', 'sku_discount', 'price_source', 'price_reason'
]

# Filter to only columns that exist
output_cols = [c for c in output_cols if c in df_results.columns]

df_results[output_cols].to_excel(OUTPUT_FILE, index=False)
print(f"\n✅ Results exported to: {OUTPUT_FILE}")
print(f"Total records: {len(df_results)}")
